Package install


In [ ]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-mistralai
#! pip install beautifulsoup4
#! pip install python-dotenv

Load Enviorment variables

In [14]:
from dotenv import load_dotenv
import os
load_dotenv()

True

Langsmith

In [15]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT'] = "Simple RAG"

API Keys

In [18]:
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
MISTAL_API_KEY = os.environ['MISTRAL_API_KEY']
#os.environ['OPENAI_API_KEY']
#os.environ['HF_TOKEN']

Rag imports

In [38]:
import bs4
import tiktoken
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain.prompts import ChatPromptTemplate


Indexing


In [ ]:
#Loading a document
#Function takes documents and loads them.
#The parse only can change, depends on the html on what you are trying to load
loader = WebBaseLoader (
    web_paths = ('https://en.wikipedia.org/wiki/Ultimate_Fighting_Championship',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer('body')
    ),
)
blog_docs =loader.load()


Splitting

In [ ]:
#Splitting the document in to chunks 
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000,
    chunk_overlap =200)

#Make splits
splits = text_splitter.split_documents(blog_docs)

Vector Storage

In [ ]:
#Uses MistralAIEmbeddings
#Embeds the chunks and saves them in the Chroma Vector DB

vectorstorage = Chroma.from_documents(documents=splits,
                                      embedding=MistralAIEmbeddings())
#Retriver object
retriver = vectorstorage.as_retriever()

c:\Users\nikol\Datamatiker\4.Semester\Python\SimpleRAG\.venv\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


Retrieval

In [35]:
docs = retriver.get_relevant_documents("Who is light weight champion")

C:\Users\nikol\AppData\Local\Temp\ipykernel_17804\3728727279.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriver.get_relevant_documents("Who is light weight champion")


In [36]:
len(docs)

4

Generation

In [39]:

#Prompt template with a context variable and a question variable
template = ''' Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

Define LLM

In [40]:
#LLM
llm = ChatMistralAI(model_name="ministral-8b-latest", temperature=0)

In [41]:
#Chain
#Connecting promt and llm togehter
chain = prompt | llm

In [42]:
#Run our chain
#Using a dictionary content and question maps to our expected keys in our template

chain.invoke({"context":docs,"question":"Who is the current heavy weight champion"})

AIMessage(content='The current heavyweight champion is Jon Jones.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4677, 'total_tokens': 4686, 'completion_tokens': 9}, 'model': 'ministral-8b-latest', 'finish_reason': 'stop'}, id='run-59d1593e-a840-41c0-a633-c6430af9302f-0', usage_metadata={'input_tokens': 4677, 'output_tokens': 9, 'total_tokens': 4686})

ANother way to chain:

In [ ]:
#Another way of doing it, looks a bit more smoother.
#We invoke with our question, that gets passed trough to our Key: Question in our Dictionary
# And it will automaticly trigger our retriver, which returns documents.
#This is automated compared to above

rag_chain = (
    {"context": retriver,"question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Who is the current heavy weight champ")

'The current heavyweight champion is Jon Jones.'